In [77]:
import pandas as pd
import numpy as np

# Feature Selection

### Load in the data

In [78]:
data = pd.read_csv('dataset/spotify_songs_restructured_cleaned.csv')
data.head()

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_genre,danceability,energy,key,loudness,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category,release_month
0,Memories - Dillon Francis Remix,Maroon 5,67.0,Memories (Dillon Francis Remix),2019-12-13,pop,0.726,0.815,11.000000,-4.969,...,0.0724,0.004210,0.35700,0.693000,99.972,162600.0,Medium Popularity,Slow,Loud,December
1,All the Time - Don Diablo Remix,Zara Larsson,70.0,All the Time (Don Diablo Remix),2019-07-05,pop,0.675,0.931,1.000000,-3.432,...,0.0794,0.000023,0.19031,0.613000,124.008,176616.0,Medium Popularity,Fast,Loud,July
2,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19,pop,0.718,0.930,7.000000,-3.778,...,0.0287,0.000009,0.20400,0.509838,121.956,169093.0,Medium Popularity,Fast,Loud,July
3,Never Really Over - R3HAB Remix,Katy Perry,62.0,Never Really Over (R3HAB Remix),2019-07-26,pop,0.449,0.856,5.387222,-4.788,...,0.1870,0.000000,0.17600,0.152000,112.648,187675.0,Medium Popularity,Moderate,Loud,July
4,Cross Me (feat. Chance the Rapper & PnB Rock) ...,Ed Sheeran,58.0,Cross Me (feat. Chance the Rapper & PnB Rock) ...,2019-06-21,pop,0.679,0.923,5.387222,-6.500,...,0.1460,0.000005,0.12400,0.752000,121.984,207894.0,Medium Popularity,Fast,Moderate,June


In [96]:
from sklearn.feature_selection import f_classif, SelectKBest, f_regression

# Create a copy of your DataFrame
# data_copy = data.copy()
# 
# # Replace all categorical columns with categorical codes
# for col in data_copy.select_dtypes(include='category').columns:
#     data_copy[col] = data_copy[col].cat.codes

X = data.drop(columns=['track_popularity']).select_dtypes(include=np.number)  # Feature set
y = data['track_popularity']  # Target
# y = data['popularity_category']  # Target

# Apply the F-test
selector = SelectKBest(f_regression, k='all') # f_regression for continuous target, f_classif for categorical target
X_new = selector.fit_transform(X, y)

# new_indices = selector.get_support(indices=True)
# best_features_df = data.iloc[:, new_indices]

# Get the scores
f_scores = selector.scores_

# list the f-score of each feature and sort them from high to low
feature_scores = pd.DataFrame({'Feature': X.columns, 'F-score': f_scores})
feature_scores = feature_scores.sort_values(by='F-score', ascending=False)
top_features = feature_scores.head(9)['Feature'].tolist()

# Step 4: Drop columns that are not in the top 9 from data_copy
data_best_features = data[['track_name', 'track_artist', 'track_album_name', 'track_popularity', 'popularity_category', 'track_album_release_date', 'playlist_genre'] + top_features]

print(feature_scores)

data_best_features.head(20)

             Feature     F-score
7   instrumentalness  418.448037
11       duration_ms  395.967208
1             energy  228.625375
6       acousticness  136.102681
0       danceability   74.680534
3           loudness   61.551568
8           liveness   49.608652
9            valence   18.166556
5        speechiness    0.945192
10             tempo    0.503744
4               mode    0.271951
2                key    0.090617


,track_name,track_artist,track_album_name,track_popularity,popularity_category,track_album_release_date,playlist_genre,instrumentalness,duration_ms,energy,acousticness,danceability,loudness,liveness,valence,speechiness
0,Memories - Dillon Francis Remix,Maroon 5,Memories (Dillon Francis Remix),67.000000,Medium Popularity,2019-12-13,pop,0.004210,162600.000000,0.815000,0.07240,0.72600,-4.969000,0.35700,0.693000,0.106983
1,All the Time - Don Diablo Remix,Zara Larsson,All the Time (Don Diablo Remix),70.000000,Medium Popularity,2019-07-05,pop,0.000023,176616.000000,0.931000,0.07940,0.67500,-3.432000,0.19031,0.613000,0.074200
2,Call You Mine - Keanu Silva Remix,The Chainsmokers,Call You Mine - The Remixes,60.000000,Medium Popularity,2019-07-19,pop,0.000009,169093.000000,0.930000,0.02870,0.71800,-3.778000,0.20400,0.509838,0.102000
3,Never Really Over - R3HAB Remix,Katy Perry,Never Really Over (R3HAB Remix),62.000000,Medium Popularity,2019-07-26,pop,0.000000,187675.000000,0.856000,0.18700,0.44900,-4.788000,0.17600,0.152000,0.062300
4,Cross Me (feat. Chance the Rapper & PnB Rock) ...,Ed Sheeran,Cross Me (feat. Chance the Rapper & PnB Rock) ...,58.000000,Medium Popularity,2019-06-21,pop,0.000005,207894.000000,0.923000,0.14600,0.67900,-6.500000,0.12400,0.752000,0.181000
5,Hate Me - R3HAB Remix,Ellie Goulding,Hate Me (R3HAB Remix),67.000000,Medium Popularity,2019-08-16,pop,0.000000,203733.000000,0.774000,0.14800,0.43700,-4.918000,0.13300,0.329000,0.106983
6,Body On My,Loud Luxury,Body On My,67.000000,Medium Popularity,2019-03-29,pop,0.000000,192507.000000,0.698913,0.03990,0.74400,-4.675000,0.37400,0.687000,0.046300
7,SOS - Laidback Luke Tribute Remix / Radio Edit,Avicii,SOS (Laidback Luke Tribute Remix),68.000000,Medium Popularity,2019-05-17,pop,0.000000,164516.000000,0.915000,0.01110,0.57200,-4.451000,0.33900,0.678000,0.062500
8,Summer Days (feat. Macklemore & Patrick Stump ...,Martin Garrix,Summer Days (feat. Macklemore & Patrick Stump ...,63.000000,Medium Popularity,2019-07-12,pop,0.001830,225829.007789,0.780000,0.00733,0.69000,-4.446000,0.07290,0.238000,0.059400
9,All My Friends - Eden Prince Remix,AJ Mitchell,All My Friends (Eden Prince Remix),60.000000,Medium Popularity,2019-05-03,pop,0.000000,173548.000000,0.901000,0.07020,0.69400,-4.322000,0.19031,0.368000,0.094800


In [100]:
from sklearn.model_selection import train_test_split

# apply train_test_split twice to create a train (60%), validation (20%) and test set (20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25)

print(f'Training set: {X_train.shape}, {y_train.shape}')
print(f'Validation set: {X_val.shape}, {y_val.shape}')
print(f'Test set: {X_test.shape}, {y_test.shape}')

Training set: (11908, 12), (11908,)
Validation set: (3970, 12), (3970,)
Test set: (3970, 12), (3970,)


In [102]:
# Example model training (Linear Regression for continuous target)
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
train_accuracy = model.score(X_train, y_train)
val_accuracy = model.score(X_val, y_val)
test_accuracy = model.score(X_test, y_test)

print(f'Train Accuracy: {train_accuracy:.2f}')
print(f'Validation Accuracy: {val_accuracy:.2f}')
print(f'Test Accuracy: {test_accuracy:.2f}')

Train Accuracy: 0.07
Validation Accuracy: 0.07
Test Accuracy: 0.05


In [103]:
# for categorical target

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Standardize the feature set
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Train a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
train_accuracy = rf_model.score(X_train_scaled, y_train)
val_accuracy = rf_model.score(X_val_scaled, y_val)
test_accuracy = rf_model.score(X_test_scaled, y_test)

print(f'Train Accuracy (RF): {train_accuracy:.2f}')
print(f'Validation Accuracy (RF): {val_accuracy:.2f}')
print(f'Test Accuracy (RF): {test_accuracy:.2f}')

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.